##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

次のPythonコードは、オペレーティングシステムライブラリを使用してファイルシステムへアクセスできるようにし、zipfileライブラリを使用してデータを解凍できるようにします。

In [ ]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

zipファイルの内容はベースディレクトリの `/tmp/horse-or-human`に解凍されます。このディレクトリには、 `horses` および `humans`サブディレクトリがあります。

要約すると、訓練セットは、「馬とはこういうものだ」、「人間とはこういうものだ」などをニューラルネットワークモデルに伝えるために使用されるデータです。 

このサンプルで注意すべきことは、画像に馬または人間のラベルを明示的には付けていないことです。以前の手書きの例を思い出してみると、「これは1」、「これは7」などのラベルを付けました。後で、画像ジェネレーターと呼ばれるものが使用されるのを見ていきますが、これはサブディレクトリから画像を読み取り、そのサブディレクトリの名前から自動的にラベルを付けるようにコード化されています。たとえば、「training」ディレクトリに「horses」ディレクトリと「humans」ディレクトリがあるとします。画像ジェネレーターが自動的に適切なラベルを画像に付けるため、コーディングの手間が省けます。

これらの各ディレクトリを定義しましょう。

In [ ]:
# 訓練用の馬の写真があるディレクトリ
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# 訓練用の人間の写真があるディレクトリ
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# 検証用の馬の写真があるディレクトリ 
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# 検証用の人間の写真があるディレクトリ　　
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)

validation_horse_hames = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)

## 小さなモデルを初めから構築する

続ける前に、モデルの定義を開始しましょう。

まず初めに、tensorflowをインポートします。

In [ ]:
import tensorflow as tf

次に、前の例と同様に畳み込み層を追加し、最終結果をフラット化して、密結合層に与えます。

最後に、密結合層を追加します。

２クラスの分類問題、つまり、*２値分類問題* に直面しているため、ネットワークは [*sigmoid* 活性化関数](https://wikipedia.org/wiki/Sigmoid_function)で終了することに注意してください。これにより、ネットワークの出力は、０と１の間の単一のスカラになり、現在の画像がクラス１（クラス０ではなく）である確率をエンコードします。

In [ ]:
model = tf.keras.models.Sequential([
    # 入力形状が、150x150、3バイトの色という望ましい画像サイズであることに注目してください
    # これは最初の畳み込みです
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # 2番目の畳み込み
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 3番目の畳み込み
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 4番目の畳み込み
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # 5番目の畳み込み
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # 結果をフラット化して、DNNに与えます
    tf.keras.layers.Flatten(),
    # 512ニューロンの隠れ層 
    tf.keras.layers.Dense(512, activation='relu'),
    # 出力ニューロンは１つのみ。結果は０から1までの値を含み、０は一方のクラス（horses）を表し、１は他方のクラス（humans）を表します
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()メソッドコールは、NN（ニューラルネットワーク）のサマリーを表示します

In [ ]:
model.summary()

"output shape" 列は、特徴マップのサイズが、連続する各層でどのように増加するかを示します。畳み込み層は、パディングにより少しだけ、特徴マップのサイズを小さくし、各プーリング層は長さを半分にします。

次に、モデルの訓練の仕様を構成します。`binary_crossentropy`損失関数を使用してモデルを訓練します。これは２値分類問題であり、最終的なアクティベーションはシグモイドだからです。（損失評価の復習については、[機械学習入門講座](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture)を参照してください。）`rmsprop`オプティマイザを`0.001`の学習率で使用します。訓練時、分類の精度を監視したいと思います。

**注**: この例では、[RMSprop最適化アルゴリズム](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp)の使用が[確率的勾配降下法](https://developers.google.com/machine-learning/glossary/#SGD)（SGD）より望ましいです。RMSpropは学習率の調整を自動化するからです。（[Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam)や[Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad)など、他の最適化アルゴリズムも訓練時の学習率を自動的に調整し、ここでも同様に良く機能します。）

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

### データの前処理

ソースフォルダの写真を読み取って、`float32`テンソルに変換し、それらを（ラベルとともに）ネットワークに与えるデータジェネレーターをセットアップしましょう。訓練用画像のジェネレーターが１つ、検証用画像のジェネレーターが１つあります。ジェネレーターはサイズが300x300の画像とそれらのラベル（２値）のバッチを生成します。

すでにご存じかもしれませんが、ニューラルネットワークに入力されるデータは、ネットワークによる処理が容易になるように、通常、何らかの方法で正規化する必要があります。（生のピクセルを畳み込みニューラルネットワークに与えるのは一般的ではありません）この例では、画像の前処理として、ピクセル値を`[0, 1]`の範囲に正規化します。（もともと、すべての値は`[0, 255]`の範囲内です）

Kerasでは、これは`keras.preprocessing.image.ImageDataGenerator`クラスにより`rescale`パラメータを使用して行うことができます。`ImageDataGenerator`クラスを使用すると、`.flow(data, labels)`または`.flow_from_directory(directory)`を介して、拡張された画像バッチ（およびそのラベル）のジェネレーターをインスタンス化できます。これらのジェネレーターは、入力としてデータジェネレーターを受け入れるKerasモデルメソッド、すなわち、`fit`、`evaluate_generator`、および`predict_generator`で使用できます。

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# すべての画像は1./255で再スケールされます
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# train_datagenジェネレーターを使用した128のバッチで訓練用画像を流します
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # これは訓練用画像のソースディレクトリです
        target_size=(150, 150),  # すべての画像は150x150にサイズ変更されます
        batch_size=128,
        # binary_crossentropy損失関数を使用するため、２値ラベルが必要です
        class_mode='binary')

# validation_datagenジェネレーターを使用した32のバッチで検証用画像を流します　　　
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # これは検証用画像のソースディレクトリです　　　
        target_size=(150, 150),  # すべての画像は150x150にサイズ変更されます
        batch_size=32,
        # binary_crossentropy損失関数を使用するため、２値ラベルが必要です
        class_mode='binary')

### 訓練
15エポックにわたって訓練しましょう。実行には数分かかることがあります。

エポックごとの値に注目してください。

「Loss（損失）」と「Accuracy（精度）」は、訓練の進捗状況を示す優れた指標です。訓練データの分類に関する推測を行い、それを既知のラベルと照合して評価し、結果を計算します。「Accuracy（精度）」は、正しい推測の割合です。 

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

### モデルの実行

では、モデルを使用して実際に実行される予測を見てみましょう。このコードでは、ファイルシステムから１つ以上のファイルを選択して、アップロードし、それらでモデルを実行して、オブジェクトが馬か人間かを推測できます。

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing.image import img_to_array, load_img

uploaded = files.upload()

for fn in uploaded.keys():
 
  # 画像の推測
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

### 中間表現の可視化

畳み込みニューラルネットワークが、どんな種類の特徴を学習するかを理解する楽しみの１つは、入力が畳み込みニューラルネットワークを通過する際にどのように変形されるか可視化することです。

訓練セットからランダムに画像を選んで、図を生成します。各行が層の出力であり、行内の各画像は、出力された特徴マップ内の特定のフィルタです。このセルを再実行して、さまざまな訓練用画像の中間表現を生成します。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# 入力として画像を受け取り、
# 最初のモデルの後に前のモデルのすべての層の中間表現を出力する
# 新しいモデルを定義しましょう
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# 訓練セットのランダムな入力画像を用意しましょう。
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(150, 150))  # これはPIL画像です
x = img_to_array(img)  # 形状(150, 150, 3)のNumpy配列
x = x.reshape((1,) + x.shape)  # 形状(1, 150, 150, 3)のNumpy配列

# 1/255で再スケールします
x /= 255

# 画像をネットワークに通して、
# この画像のすべての中間表現を取得しましょう。
successive_feature_maps = visualization_model.predict(x)

# これらは層の名前なので、プロットの一部として使用できます
layer_names = [layer.name for layer in model.layers]

# 表現を表示しましょう
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # これは畳み込み層とmaxpool層についてのみ行い、全結合層については行いません
    n_features = feature_map.shape[-1]  # 特徴マップ内の特徴の数
    # 特徴マップの形状は (1, size, size, n_features)です
    size = feature_map.shape[1]
    # 画像をこのマトリックスに並べます
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # 特徴を後処理して、見やすくします
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      #  各フィルタをこの大きな水平行列に並べます
      display_grid[:, i * size : (i + 1) * size] = x
    # 行列を表示します
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

ご覧のように、画像の生のピクセルから、徐々に抽象的でコンパクトな表現になっていきます。表現のダウンストリームは、ネットワークが注意を払うものを強調し始め、「アクティベート」される特徴が徐々に少なくなり、ほとんどがゼロにセットされます。これを「スパース性」といいます。表現のスパース性は、ディープラーニングの重要な特徴です。


これらの表現は、画像の元のピクセルに関して、ますます少ない情報しか伝えませんが、画像のクラスに関しては、より洗練された情報を伝えます。畳み込みニューラルネットワーク（または一般に、深層ネットワーク）は、情報を蒸留するパイプラインと考えることができます。

## クリーンアップ

次の練習問題を行う前に、次のセルを実行してカーネルを終了し、メモリリソースを解放してください。

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)